In [8]:
from draughts import Draughts
import random

class RandomPlayer:

    def move(self, board, player_id, valid_moves):
        a = random.choice(valid_moves)
        return a

In [9]:
game =Draughts(RandomPlayer(), RandomPlayer())

game.play()

  0 1 2 3 4 5 6 7
0   w   w   w   w 
1 w   w   w   w   
2   w   w   w   w 
3                 
4                 
5 b   b   b   b   
6   b   b   b   b 
7 b   b   b   b   
Player 2 wins! Player 1 has no valid moves.


'loss'